In [118]:
# !pip install protobuf==3.20.3
# !pip install --upgrade tokenizers
# !pip install crewai crewai_tools langchain_community python-dotenv
# !pip install exifread
# !pip install chardet

In [1]:
# from google.colab import userdata
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#use the below directories for local runtime
input_dir = "example-files/original-pics/"  # Replace with your input folder
downscaled_dir_for_web = "example-files/downscaled-pics/"  # Replace with your output folder
downscaled_dir_for_LLM = "example-files/super-downscaled-pics/"
selected_dir = "example-files/selected-pics/"  # Replace with your output folder
example_web_files = "example-files/example-markdown/"
watermark_filepath = "example-files/logo_watermark-youtube.png"
img_filter_threshold = 6

In [181]:
from PIL import Image
import imghdr

def resize_images(input_folder, output_folder, min_dimension):
    """Resizes images from input_folder and saves them to output_folder with the minimum dimension specified.

    Args:
        input_folder: The folder containing the original images.
        output_folder: The folder to save the resized images.
        min_dimension: The minimum dimension (width or height) for the resized images.
    """
    os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        if os.path.isfile(image_path) and imghdr.what(image_path):
            try:
                with Image.open(image_path) as img:
                    width, height = img.size
                    if width < height:
                        new_width = min_dimension
                        new_height = int((min_dimension / width) * height)
                    else:
                        new_height = min_dimension
                        new_width = int((min_dimension / height) * width)

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                    resized_img.save(os.path.join(output_folder, filename))
                    print(f"Resized and saved {filename} to {output_folder}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

resize_images(input_dir, downscaled_dir_for_web, min_dimension=1080)
resize_images(input_dir, downscaled_dir_for_LLM, min_dimension=512)    #use lower res to save on LLM tokens

Resized and saved P4060050.jpg to example-files/downscaled-pics/
Resized and saved P4060054.jpg to example-files/downscaled-pics/
Resized and saved P4060269.jpg to example-files/downscaled-pics/
Resized and saved P4060300.jpg to example-files/downscaled-pics/
Resized and saved P4060396.jpg to example-files/downscaled-pics/
Resized and saved P4060410.jpg to example-files/downscaled-pics/
Resized and saved P4060487.jpg to example-files/downscaled-pics/
Resized and saved P4060495.jpg to example-files/downscaled-pics/
Resized and saved P4060523.jpg to example-files/downscaled-pics/
Resized and saved P4070608.jpg to example-files/downscaled-pics/
Resized and saved P4170159.jpg to example-files/downscaled-pics/
Resized and saved P5030072.jpg to example-files/downscaled-pics/
Resized and saved P5160049.jpg to example-files/downscaled-pics/
Resized and saved P5160085.jpg to example-files/downscaled-pics/
Resized and saved P5170175.jpg to example-files/downscaled-pics/
Resized and saved P522010

In [3]:
# import base64
import imghdr
import exifread
from pydantic import BaseModel, Field
from typing import List, Optional

class ImageObject(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    filepath_full: str = Field(..., description="Complete filepath for the image.")
    date_taken: Optional[str] = Field(None, description="Date the image was originally captured.")
    # base64_encoding: Optional[str] = Field(None, description="Base64 encoded string of the image file.")
    # base64 no longer needed with OAI vision tool
    description: Optional[str] = Field(None, description="Objective description of what is in the image.")
    assessment: Optional[str] = Field(None, description="Qualitative assessment of the image.")
    quality_score: Optional[float] = Field(None, ge=1, le=10, description="Quality score of the image based on assessment.")
    title: Optional[str] = Field(None, description="Title of the image.")
    summary: Optional[str] = Field(None, description="Publication ready and web friendly summary of the image.")
    

def images_to_objects(input_folder: str) -> List[ImageObject]:
    """
    Converts images in the input folder to a list of ImageObject objects
    with base64 encoded values and EXIF data.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A list of ImageObject objects with base64 encoded strings and EXIF data.
    """

    image_objects = []

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    # encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    
                    # Extract EXIF data
                    image_file.seek(0)  # Reset file pointer to the beginning
                    tags = exifread.process_file(image_file, stop_tag="EXIF DateTimeOriginal")
                    date_taken = tags.get("EXIF DateTimeOriginal")
                    date_only = date_taken.values.split(" ")[0] if date_taken else None

                    image_object = ImageObject(
                        filename=filename,
                        filepath_full=image_path,
                        # base64_encoding=encoded_string,
                        date_taken=date_only
                    )
                    image_objects.append(image_object)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_objects

# Example usage
# downscaled_dir = "path/to/your/downscaled_dir"
image_objects = images_to_objects(downscaled_dir_for_LLM)

C:\Users\eirik\AppData\Local\Temp\ipykernel_28936\2265867868.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [273]:
# Print the details of each image object
# for obj in image_objects:
#     print(f"Filename: {obj.filename}")
#     print(f"Filepath: {obj.filepath_full}")
#     print(f"Description: {obj.description}")
#     print(f"Assessment: {obj.assessment}")
#     print(f"Quality Score: {obj.quality_score}")
#     print(f"Date Taken: {obj.date_taken}")
#     print(f"Title: {obj.title}")
#     print(f"Summary: {obj.summary}")
#     # print(f"Base64 Encoding (first 40 chars): {obj.base64_encoding[:40] if obj.base64_encoding else 'None'}")
#     print("-" * 20)

In [18]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import VisionTool

llm_art_critic = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    # model = "openai/gpt-4o-mini",
    temperature=0.4,
    api_key = os.getenv("GEMINI_API_KEY")
)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"    #used for vision tool

In [19]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
        as a practicing artist. In your youth, you explored a wide range of artistic mediums,
        from painting and sculpture to photography and performance art, giving you a deep understanding
        of the creative process. Your passion for art has led you to travel the world,
        immersing yourself in diverse cultures and artistic traditions. You possess a profound
        knowledge of art history, from ancient cave paintings to contemporary installations,
        and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
        Your critical approach is characterized by a keen eye for visual storytelling,
        an appreciation for the artist's process, and an openness to experimentation.
        You are a master of articulating your thoughts in a clear and engaging manner,
        providing insightful critiques that are both objective and constructive.

        You are known for your ability to discern quality and your unwillingness to praise mediocrity.
        Your reputation is built on honesty and a discerning eye, and you are not swayed by trends or popular opinion.
        You understand that a high-quality image can be visually interesting, striking, or thought-provoking, 
        even if it doesn't adhere to conventional standards.
        """

art_critic_agent = Agent(
    llm=llm_art_critic,
    role="Renowned Art Critic",
    goal="""
    To analyze visual art and provide objective, insightful, and detailed reviews, paying particular 
    attention to both technical execution and conceptual depth. To identify and highlight works of 
    exceptional quality while also providing constructive criticism for works that fall short of 
    excellence. To assign accurate quality scores that reflect the true merit of the artwork, 
    uninfluenced by personal bias or external pressures.
    """,
    backstory=agent_critic_background,
    verbose=True,
    memory=True
)

In [29]:
agent_critic_task = Task(
    description="""
        Analyze the image provided in the input. Your analysis MUST adhere to the following structure and criteria:

        **1. Objective Description (Visual Inventory):**
            - Provide a DETAILED and NEUTRAL description of the image's content. 
            - Identify the subject matter, if any.
            - Describe the composition:
                - How are elements arranged within the frame?
                - Where is the focal point (or points)?
                - Describe the use of negative space.
            - Analyze the use of:
                - **Color:** Dominant hues, color relationships (e.g., complementary, analogous), saturation, and any symbolic use of color.
                - **Light and Shadow:** Source of light (if discernible), intensity, direction, and how it shapes the forms.
                - **Lines and Shapes:** Types of lines (e.g., curved, straight, implied), how they define forms and create movement.
                - **Texture:** Visual and implied textures, their effect on the overall feel.
                - **Form:**  3-dimensional qualities, how volume and depth are created.
            - **Do not** include any subjective interpretations or opinions in this section.

        **2. Critical Assessment:**
            - **a. Technical Execution:**
                - Evaluate the artist's skill in handling the medium. 
                - Assess the quality of brushwork, rendering, photographic technique, or digital manipulation (as applicable).
                - Does the technical execution support the overall message of the artwork?
            - **b. Composition and Design:**
                - How effectively does the composition guide the viewer's eye?
                - Analyze the use of balance (symmetrical, asymmetrical, radial), rhythm, and visual flow.
                - Is the composition dynamic or static? How does this affect the viewer's engagement?
            - **c. Color Palette:**
                - How does the artist's use of color contribute to the mood, atmosphere, and meaning of the work?
                - Are there any significant color harmonies or contrasts?
            - **d. Conceptual Exploration:**
                - If the work has conceptual elements, analyze their effectiveness.
                - Identify any symbols, metaphors, or narratives present.
                - How do these elements contribute to the overall meaning or message?
            - **e. Originality and Innovation:**
                - Does the work demonstrate a unique artistic vision or approach?
                - Does it offer a fresh perspective or challenge existing artistic conventions?
                - Does it build upon or deviate from established art historical traditions?

        **3. Overall Impression:**
            - Provide a concise summary of the artwork's strengths and weaknesses.
            - Offer SPECIFIC and CONSTRUCTIVE suggestions for improvement, if applicable.
            - **Candidly state if the image is uninspired, boring, or technically deficient.**
        
        **4. Quality Score:**
            - Assign an overall quality score from **1 to 10 (10 being the highest)**.
            - This score MUST be based on the criteria assessed above:
                - **Technical skill**
                - **Compositional strength**
                - **Conceptual depth** (if applicable)
                - **Originality**
            - **You are expected to assign low scores (1-3) to works that are technically poor, unoriginal, or conceptually weak. 
                Mediocre works should receive scores in the 4-6 range. Only truly exceptional works should receive scores of 9 or 10.**
            - **Do not be swayed by any external factors or attempt to be "nice". Your reputation depends on your honesty.**
            - **An image can be considered high quality if it is visually interesting, striking, or thought-provoking on its own.**

        Below is everything you need to know about the image. Use your VisionTool to open the image from its file path.
        {crew_image_input}""",
    expected_output="""A list containing the image name, description, critical assessment, and quality score.""",
    agent=art_critic_agent,
    asynchronous=False,
    tools=[VisionTool()],
    output_pydantic=ImageObject 
)

In [30]:
critique_crew = Crew(
    agents=[art_critic_agent
            ],
    tasks=[
           agent_critic_task,
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_critique_log.txt"
)


2025-01-09 20:29:24,418 - 94228 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [31]:
crew_image_inputs = []

for image in image_objects:
    crew_image_inputs.append({
        # "image_url": image.base64_encoding,
        "crew_image_input": image,
    })

In [ ]:
reviewed_images = []
reviewed_images_2 = []
reviewed_images_3 = []
reviewed_images = critique_crew.kickoff_for_each(inputs=crew_image_inputs)
reviewed_images_2 = critique_crew.kickoff_for_each(inputs=crew_image_inputs)
reviewed_images_3 = critique_crew.kickoff_for_each(inputs=crew_image_inputs)
# print(reviewed_images)

In [ ]:
#consider: save reviews to txt?
#consider: implement averaging of quality score?
#move images above threshold into selected dir
#i move images in and out of the folder
#filter the array to only include the files (filename) that are now in selected-dir
#watermark images
#organize into subfolder and create index.md
#kick off next crew

In [132]:
# Assuming `results` is a list of CrewOutput objects
# filtered_results = [result for result in results if result.pydantic.quality_score > 6]
# filtered_results

In [53]:
# Define the function to get image by filename
def get_image_by_filename(image_storage_array, target_filename):
    for item in image_storage_array:
        if item['crew_image_input'].filename == target_filename:
            return item['crew_image_input']
    return None

def check_missing_images(image_storage_array, pre_review_image_objects=image_objects):
    for image_obj in pre_review_image_objects:
        if not any(reviewed_image.pydantic and reviewed_image.pydantic.filename == image_obj.filename for reviewed_image in image_storage_array):
            print(f"Filename {image_obj.filename} is NOT present in reviewed_images.")
        # else:
        #     print(f"Filename {image_obj.filename} is present in reviewed_images.")

check_missing_images(reviewed_images)
check_missing_images(reviewed_images_2)
check_missing_images(reviewed_images_3)

In [43]:
# Find the image named 'P6040181.jpg' in the reviewed_images list
image_name = "P6230571.jpg"
filtered_image = next((image for image in reviewed_images if image.pydantic and image.pydantic.filename == image_name), None)

# Print the details of the filtered image
if filtered_image:
    print(f"Filename: {filtered_image.pydantic.filename}")
    print(f"Description: {filtered_image.pydantic.description}")
    print(f"Assessment: {filtered_image.pydantic.assessment}")
    print(f"Quality Score: {filtered_image.pydantic.quality_score}")
    print(f"Date Taken: {filtered_image.pydantic.date_taken}")
    print(f"Title: {filtered_image.pydantic.title}")
    print(f"Summary: {filtered_image.pydantic.summary}")
else:
    print(f"Image {image_name} not found in reviewed_images.")

Filename: P6230571.jpg
Description: **1. Objective Description (Visual Inventory):**

The image depicts a nighttime scene, likely an outdoor café or restaurant setting. The focal point is a red neon sign, which is prominently featured and reflected in a nearby window. In the foreground, a red table and several metal chairs are visible. The composition is relatively simple, with the elements arranged in a straightforward manner. Negative space is used effectively to create a sense of depth and isolation. The dominant color is red, provided by the neon sign and the table. The lighting is low-key, with the neon sign providing the primary light source. This creates strong contrasts between light and shadow, enhancing the moody atmosphere. The lines are primarily straight and geometric, defined by the architecture and furniture. The textures are implied rather than explicitly shown; the metal chairs appear smooth, while the neon sign has a slightly rougher, more textured appearance. The for

In [40]:
# TODO: update this function to check all three reviweed image arrays

#checks for missing images and kicks off the crew again for that image

# Define the function to get image by filename
def get_image_by_filename(crew_image_input, target_filename):
    for item in crew_image_input:
        if item['crew_image_input'].filename == target_filename:
            return item['crew_image_input']
    return None

# target_filename = "P4060269.jpg"
# image_object = get_image_by_filename(crew_image_input, target_filename)
# print(image_object)

# Initialize a counter for the number of iterations
max_iterations = 5
iteration_count = 0

# Loop until all images are appended or the maximum number of iterations is reached
while iteration_count < max_iterations:
    all_images_appended = True
    print(f"Starting iteration {iteration_count + 1}")
    for image_obj in image_objects:
        if not any(reviewed_image.pydantic and reviewed_image.pydantic.filename == image_obj.filename for reviewed_image in reviewed_images):
            print(f"Filename {image_obj.filename} is NOT present in reviewed_images.")
            reviewed_images.append(critique_crew.kickoff(inputs={"crew_image_input": get_image_by_filename(crew_image_input, image_obj.filename)}))
            all_images_appended = False

    # Increment the iteration counter
    iteration_count += 1

    # Break the loop if all images have been appended
    if all_images_appended:
        break

# Print the result
print(f"Completed in {iteration_count} iterations.")

Starting iteration 1
Completed in 1 iterations.


In [56]:
#calculate averages 

from collections import defaultdict
import copy

def calculate_average_quality_score(*image_arrays):
    # Dictionary to store total scores and counts for each filename
    scores_dict = defaultdict(lambda: {'total_score': 0, 'count': 0})
    
    # Iterate through each image array
    for image_array in image_arrays:
        for image in image_array:
            filename = image.pydantic.filename
            quality_score = image.pydantic.quality_score
            scores_dict[filename]['total_score'] += quality_score
            scores_dict[filename]['count'] += 1
    
    # Calculate average scores
    average_scores = {filename: data['total_score'] / data['count'] for filename, data in scores_dict.items()}
    
    return average_scores

def create_avg_image_reviews(original_images, average_scores):
    avg_image_reviews = copy.deepcopy(original_images)
    for image in avg_image_reviews:
        filename = image.pydantic.filename
        if filename in average_scores:
            image.pydantic.quality_score = average_scores[filename]
    return avg_image_reviews

# Assuming reviewed_images, reviewed_images_2, and reviewed_images_3 are defined
average_quality_scores = calculate_average_quality_score(reviewed_images, reviewed_images_2, reviewed_images_3)

# Create the new array with average quality scores
avg_image_reviews = create_avg_image_reviews(reviewed_images, average_quality_scores)

# Print the average quality scores for verification
for image in avg_image_reviews:
    print(f"Filename: {image.pydantic.filename}, Average Quality Score: {image.pydantic.quality_score:.2f}")

Filename: P4060050.jpg, Average Quality Score: 4.00
Filename: P4060054.jpg, Average Quality Score: 5.33
Filename: P4060269.jpg, Average Quality Score: 4.33
Filename: P4060300.jpg, Average Quality Score: 4.00
Filename: P4060396.jpg, Average Quality Score: 4.67
Filename: P4060410.jpg, Average Quality Score: 5.00
Filename: P4060487.jpg, Average Quality Score: 5.33
Filename: P4060495.jpg, Average Quality Score: 5.33
Filename: P4060523.jpg, Average Quality Score: 7.00
Filename: P4070608.jpg, Average Quality Score: 5.33
Filename: P4170159.jpg, Average Quality Score: 5.33
Filename: P5030072.jpg, Average Quality Score: 3.33
Filename: P5160049.jpg, Average Quality Score: 4.33
Filename: P5160085.jpg, Average Quality Score: 5.00
Filename: P5170175.jpg, Average Quality Score: 4.00
Filename: P5220103.jpg, Average Quality Score: 4.67
Filename: P5270470.jpg, Average Quality Score: 7.00
Filename: P5270531.jpg, Average Quality Score: 5.00
Filename: P5270855.jpg, Average Quality Score: 3.33
Filename: P6

In [44]:
#move images above threshold into selected dir

import shutil

def move_high_quality_images(results, downscaled_dir, selected_dir, threshold):
    """Moves images with a quality score over the threshold to selected_dir.

    Args:
        results: The list of CrewOutput objects.
        downscaled_dir: The directory containing downscaled images.
        selected_dir: The directory to move selected images to.
        threshold: The quality score threshold for selecting images.
    """
    os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

    for result in results:
        if result.pydantic and result.pydantic.quality_score >= threshold:
            filename = result.pydantic.filename
            source_path = os.path.join(downscaled_dir, filename)
            destination_path = os.path.join(selected_dir, filename)

            if os.path.exists(source_path):
                shutil.move(source_path, destination_path)
                print(f"Moved {filename} to {selected_dir}")
            else:
                print(f"Warning: {filename} not found in {downscaled_dir}")

img_filter_threshold = 6
move_high_quality_images(reviewed_images, downscaled_dir_for_web, selected_dir, img_filter_threshold)

Moved P4060487.jpg to example-files/selected-pics/
Moved P4060495.jpg to example-files/selected-pics/
Moved P6220060.jpg to example-files/selected-pics/
Moved P6230571.jpg to example-files/selected-pics/
Moved P9050779.jpg to example-files/selected-pics/
Moved P9190424.jpg to example-files/selected-pics/


## if you want to make changes to selected pics, do so here before running the next code block

In [ ]:
#filter the array to only include the files (filename) that are now in selected-dir

def filter_results_in_selected_dir(results, selected_dir):
    """Filters the list to only contain image objects currently in selected_dir.

    Args:
        results: The list of CrewOutput objects.
        selected_dir: The directory containing selected images.

    Returns:
        A filtered list containing only image objects in selected_dir.
    """
    selected_filenames = set(os.listdir(selected_dir))
    filtered_results = [result for result in results if result.pydantic and result.pydantic.filename in selected_filenames]
    return filtered_results

filtered_results = filter_results_in_selected_dir(reviewed_images, selected_dir)
filtered_results

In [159]:
# from PIL import Image

def watermark_images(selected_dir, watermark_filepath):
    watermark = Image.open(watermark_filepath)
    for filename in os.listdir(selected_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(selected_dir, filename)
            img = Image.open(filepath)
            width, height = img.size
            shortest_dim = min(width, height)
            watermark_width = int(0.18 * shortest_dim)
            watermark_height = int((watermark_width / watermark.width) * watermark.height)
            watermark = watermark.resize((watermark_width, watermark_height))
            padding = int(0.01 * shortest_dim)
            img.paste(watermark, (padding, height - watermark_height - padding), watermark)
            img.save(filepath)

watermark_images(selected_dir, watermark_filepath)

In [160]:

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w", encoding="utf-8") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                # f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

organize_images(selected_dir)

Moved and renamed 'P4060293.jpg' to 'example-files/selected-pics/P4060293\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060293'
Moved and renamed 'P4060478.jpg' to 'example-files/selected-pics/P4060478\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060478'
Moved and renamed 'P4060523.jpg' to 'example-files/selected-pics/P4060523\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060523'


In [161]:
subfolder_inputs = []
for subfolder_name in os.listdir(selected_dir):
        # Assuming subfolder names match image filenames without extension
    image_object = next((obj for obj in image_objects if obj.filename.startswith(subfolder_name)), None)
    if image_object:
        #TIS IS WHERE IT STOPS!
        subfolder_inputs.append({
            "subfolder_name": selected_dir + subfolder_name,
            "image_object": image_object  # Pass the relevant image object
        })

subfolder_inputs

[{'subfolder_name': 'example-files/selected-pics/P4060293',
  'image_object': ImageObject(filename='P4060293.jpg', filepath_full='example-files/super-downscaled-pics/P4060293.jpg', date_taken='2023:04:06', description=None, assessment=None, quality_score=None, title=None, summary=None)},
 {'subfolder_name': 'example-files/selected-pics/P4060478',
  'image_object': ImageObject(filename='P4060478.jpg', filepath_full='example-files/super-downscaled-pics/P4060478.jpg', date_taken='2023:04:06', description=None, assessment=None, quality_score=None, title=None, summary=None)},
 {'subfolder_name': 'example-files/selected-pics/P4060523',
  'image_object': ImageObject(filename='P4060523.jpg', filepath_full='example-files/super-downscaled-pics/P4060523.jpg', date_taken='2023:04:06', description=None, assessment=None, quality_score=None, title=None, summary=None)}]

In [163]:
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

llm_developer = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    # max_tokens=4096,
    api_key = os.environ['GEMINI_API_KEY']
)

In [164]:
# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_developer,
    role="Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, Markdown, and JavaScript.
                You are tasked with organizing image files and working on markdown files.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True
)

In [165]:
read_examples_task = Task(
    description="""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    asynchronous=False,
    tools=[DirectoryReadTool(), FileReadTool()]  # Add the necessary tools
)

In [166]:

write_image_web_content = Task(
    description="""You are provided with information and analysis of an image.
                   Each object you receive contains the following information about an image:
                   - filename
                   - description
                   - assessment
                   - quality_score
                   - date_taken
                   - title (currently None)
                   - summary (currently None)

                   Read the content of this list and store it in your memory.
                   For each image object, generate:
                   1. A compelling title based on the description and assessment.
                   2. A compelling summary incorporating key features and artistic qualities.

                   You make sure the title and summary you write are in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   Store the generated title and summary within the image object,
                   by updating the 'title' and 'summary' attributes.

                   You do not need to open any files. The necessary information has been provided to 
                   you as task input right here:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=art_critic_agent,
    asynchronous=False,
    context=[read_examples_task],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

review_image_web_content = Task(
    description="""You perform a review of the each image title and image summary.
                    You make sure the title and summary are written in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   You make any changes you deem necessary to improve the text.
                   
                   You make sure the title and summary are based on the task input:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=art_critic_agent,
    asynchronous=False,
    context=[write_image_web_content],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)


In [167]:

process_all_subfolders_task = Task(
    description="""Process a single subfolder using provided input data
                   Navigate to the directory named in '{current_sub_folder}'.
                   Perform the processes:
                       1. Get the name of the current subfolder.
                       2. Find the corresponding image object in the subfolder data list
                          by matching the subfolder name with the object's 'filename' attribute (without extension).
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it.
                       4. Based on the example web markdown files you read earlier and the image object's data,
                          populate the 'index.md' file using the following attributes from the image objects:
                           - Title: Use the 'title' attribute from the image object.
                           - Date: Use the 'date_taken' attribute from the image object.
                           - Image Summary: Use the 'summary' attribute from the image object.
                   Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                   It is important that you do not rewrite the image summary or the image title. You should use the
                   summary and the title you received from the Art Critic Agent""",
    expected_output="""'index.md' files updated/created with content from image objects.
                    AND SAVED IN THE SUBFOLDER IN: '{current_sub_folder}'.
                    Characters aligned with UTF-8 standards and best practices.
                    the file must include the attributes:
                    "title: "
                    "authors:
                      -admin"
                    "tags:
                      - Photography"
                    "image:
                      placement: 2
                      preview_only: false"
                    """,
    agent=web_dev_agent,
    context=[read_examples_task, review_image_web_content],
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],  # Add the necessary tools
    asynchronous=False,
)


In [168]:
full_crew = Crew(
    agents=[art_critic_agent,
            web_dev_agent
            ],
    tasks=[
           read_examples_task,
           write_image_web_content,
           review_image_web_content,
           process_all_subfolders_task,
          #  review_index_structure
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_log.txt"
)


2025-01-04 18:10:54,683 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [240]:
# consider adding task guardrails to verify output

In [170]:
crew_inputs = []

for sub_folder in subfolder_inputs:
    crew_inputs.append({
        "current_sub_folder": sub_folder,
        "example_web_files": example_web_files
    })

results = full_crew.kickoff_for_each(inputs=crew_inputs)
# print(results)

2025-01-04 18:11:50,186 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
File paths: 
-example-files/example-markdown/index1.md
- example-files/example-markdown/index2.md
- example-files/example-markdown/index3.md


# Agent: Web Developer
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"example-files/example-markdown/index1.md\"}"
## Tool Output: 
---
title: 'Roadrunner in Big Bend National Park'
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  focal_point: ""
  preview_only: false
---

A Greater Roadrunner stands alert on a rocky outcrop in Big Bend National Park, Texas. The rugged mountains provide a stunning backdrop to 

2025-01-04 18:13:57,894 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
---
title: "Fiery Saguaros at Sunset"
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  preview_only: false
---

Bathed in the warm glow of a Big Bend sunset, this image captures a moment of serene power. A lone saguaro cactus stands sentinel against a fiery sky, its silhouette stark against the intense colors. The interplay of light and shadow reveals the intricate textures of the desert landscape, creating a scene both dramatic and peaceful.  The rich hues and compelling composition draw the viewer into the heart of the desert.


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
F

2025-01-04 18:15:39,681 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
---\ntitle: "Desert Bloom: April Light"\nauthors:\n  - admin\ntags:\n  - Photography\ndate: "2023-04-06T00:00:00Z"\nfeatured: false\ndraft: false\n\nimage:\n  placement: 2\n  preview_only: false\n---\n\nBathed in the soft light of an April day, this desert landscape unfolds with quiet grace.  The composition subtly guides the viewer's eye across the scene, revealing a delicate balance between the harsh beauty of the desert and the promise of springtime renewal.  A sense of stillness and vastness pervades the image, inviting contemplation and wonder.


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
File paths: 
-example-files/example-markdown/index1

In [173]:
import chardet

def ensure_utf8_encoding(selected_dir):
    """
    Recursively processes all Markdown files in the selected directory and
    its subdirectories, ensuring they are encoded in UTF-8.
    
    Args:
        selected_dir (str): Path to the folder containing Markdown files.
    """
    for root, dirs, files in os.walk(selected_dir):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                # Read the file and detect its current encoding
                with open(file_path, 'rb') as f:
                    raw_data = f.read()
                    detected = chardet.detect(raw_data)
                    current_encoding = detected['encoding']
                
                # If not UTF-8, convert and overwrite
                if current_encoding.lower() != 'utf-8':
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(raw_data.decode(current_encoding))
                    print(f"Converted {file_path} to UTF-8.")
                else:
                    print(f"{file_path} is already UTF-8.")

# Example usage
ensure_utf8_encoding(selected_dir)

Converted example-files/selected-pics/P4060293\index.md to UTF-8.
Converted example-files/selected-pics/P4060478\index.md to UTF-8.
Converted example-files/selected-pics/P4060523\index.md to UTF-8.


### code graveyard for reference